In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.feature_selection import SelectKBest, SelectPercentile, mutual_info_classif
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedStratifiedKFold

from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.svm import SVC

In [2]:
df = pd.read_csv('selected_features.csv')
df

,verb_count,proper_noun_count,emotiveness,compound,neutral,label
0,2,2,0.200000,-0.4939,0.656,0
1,1,3,1.000000,0.3818,0.755,0
2,1,5,0.500000,0.0000,1.000,0
3,1,2,0.200000,0.0000,1.000,0
4,2,1,0.600000,0.2449,0.835,0
...,...,...,...,...,...,...
44868,1,3,0.666667,-0.3818,0.580,1
44869,1,3,0.000000,0.5267,0.764,1
44870,1,3,0.500000,0.4404,0.838,1
44871,3,2,0.250000,0.0000,1.000,1


In [3]:
df = df.sample(frac = 1, random_state=42)
X = df.loc[:,['verb_count',
 'proper_noun_count',
 'emotiveness',
 'compound',
 'neutral']]
y=df.label

#Split X & y into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y,
    test_size=0.2, shuffle = True, random_state = 8)
#Split remaining train into train and  validation
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, 
    test_size=0.25, random_state= 8) # 0.25 x 0.8 = 0.2


print("X_train shape: {} \n".format(X_train.shape))
print("X_test shape: {} \n" .format(X_test.shape))
print("X_val shape: {} \n".format(X_val.shape))
print("y_train shape: {}\n".format(y_train.shape))
print("y_test shape: {}\n".format(y_test.shape))
print("y_val shape: {}\n".format(y_val.shape))


# Scale the featrues, you are fitting and transforming the train features but not the test/val. Testing data must remain unseen by the model. If you fit the testing data then your model learned on the testing data
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_val = sc.transform(X_val)
X_test = sc.transform(X_test)

X_train shape: (26923, 5) 

X_test shape: (8975, 5) 

X_val shape: (8975, 5) 

y_train shape: (26923,)

y_test shape: (8975,)

y_val shape: (8975,)



In [4]:
svm =SVC()

In [5]:
X.size

224365

In [6]:
y.size

44873

In [7]:
svm.fit(X_train, y_train)

SVC()

In [8]:
y_pred_val = svm.predict(X_val)
print(classification_report(y_pred_val, y_val))

              precision    recall  f1-score   support

           0       0.87      0.76      0.81      4844
           1       0.75      0.87      0.81      4131

    accuracy                           0.81      8975
   macro avg       0.81      0.81      0.81      8975
weighted avg       0.82      0.81      0.81      8975



In [9]:
svm.get_params()

{'C': 1.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [11]:
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

kernel = ['rbf','linear', 'sigmoid', 'poly']
gamma = ['scale', 'auto']

param_grid = dict(kernel=kernel, gamma=gamma)

gridCV = GridSearchCV(svm, param_grid, scoring='accuracy', n_jobs=-1, cv=cv, error_score=0, verbose=1)

grid_fit=gridCV.fit(X_train, y_train)

#Best parameters and accuracy
print("Best: %f using %s" % (grid_fit.best_score_, grid_fit.best_params_))

Fitting 30 folds for each of 8 candidates, totalling 240 fits


KeyboardInterrupt: 

In [70]:
y_pred_val = gridCV.predict(X_val)
print(classification_report(y_pred_val, y_val))

              precision    recall  f1-score   support

           0       0.88      0.75      0.81      5088
           1       0.73      0.87      0.79      3887

    accuracy                           0.80      8975
   macro avg       0.81      0.81      0.80      8975
weighted avg       0.82      0.80      0.80      8975



In [71]:
pl = pd.read_csv('political news for cv')
gp = pd.read_csv('gossip news for cv')
gp = gp.sample(frac = 1, random_state=42)
pl = pl.sample(frac = 1, random_state=42)

In [72]:
X_gp = gp.loc[:,['verb_count',
 'proper_noun_count',
 'emotiveness',
 'compound',
 'neutral']]
y_gp = gp.label


X_gp = sc.transform(X_gp)

In [73]:
y_pred_gp = gridCV.predict(X_gp)
print(classification_report(y_pred_gp, y_gp))

              precision    recall  f1-score   support

           0       0.68      0.79      0.73     14241
           1       0.42      0.29      0.34      7381

    accuracy                           0.62     21622
   macro avg       0.55      0.54      0.54     21622
weighted avg       0.59      0.62      0.60     21622



In [62]:
X_pl = pl.loc[:,['verb_count',
 'proper_noun_count',
 'emotiveness',
 'compound',
 'neutral']]
y_pl = pl.label


sc = StandardScaler()
X_pl = sc.transform(X_pl)

In [63]:
y_pred_pl = gridCV.predict(X_pl)
print(classification_report(y_pred_pl, y_pl))

              precision    recall  f1-score   support

           0       0.73      0.75      0.74       437
           1       0.70      0.68      0.69       373

    accuracy                           0.72       810
   macro avg       0.72      0.72      0.72       810
weighted avg       0.72      0.72      0.72       810

